In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('output/lifts.csv',index_col=0)

In [2]:
import re
df_posts=pd.read_csv('output/results_vDP.csv', header=None)
s_post=df_posts[2]
s_post=s_post[s_post.notnull()]

brand_model = pd.read_csv('car_models.csv')

s_brand=brand_model['Brand'].map(lambda x:x.lower())
s_brand=s_brand.map(lambda x:x.replace('.','').replace(',','').replace('-',' '))
s_model=brand_model['Model'].map(lambda x:x.lower())
s_model=s_model.map(lambda x:x.replace('?',''))

In [3]:
df_posts[2]

0       I personally think that with a few tweaks the ...
1       I am debating a new purchase and these two are...
2       Great handling, RWD, excellent engine and the ...
3       And no manual tranny. That may not matter to y...
4       One beauty of BMW 3 Series is that there are s...
                              ...                        
4995        Because the new 3 series is the class leader?
4996    Yes, you'll be fine with RWD, even north of Da...
4997    PEARL, thanks for the tips! I usually don't go...
4998    I agree that the RWD decision is a non-issue i...
4999    >But I have to disagree with remark regarding ...
Name: 2, Length: 5000, dtype: object

In [4]:
s_brand

0      acura
1      acura
2      acura
3      acura
4      acura
       ...  
525    volvo
526    volvo
527    volvo
528    volvo
529    volvo
Name: Brand, Length: 530, dtype: object

In [5]:
import re
s_post_converted=s_post.map(lambda x:x.lower())

for i,m in enumerate(s_model):
    s_post_converted=s_post_converted.map(lambda x:re.sub(str(m), s_brand[i],str(x)))

In [6]:
s_post

0       I personally think that with a few tweaks the ...
1       I am debating a new purchase and these two are...
2       Great handling, RWD, excellent engine and the ...
3       And no manual tranny. That may not matter to y...
4       One beauty of BMW 3 Series is that there are s...
                              ...                        
4995        Because the new 3 series is the class leader?
4996    Yes, you'll be fine with RWD, even north of Da...
4997    PEARL, thanks for the tips! I usually don't go...
4998    I agree that the RWD decision is a non-issue i...
4999    >But I have to disagree with remark regarding ...
Name: 2, Length: 5000, dtype: object

In [7]:
s_post_converted
s_post_converted.to_csv('output/commentsfile.csv')

In [8]:
brands=s_brand.unique()
brands

array(['acura', 'audi', 'bmw', 'buick', 'cadillac', 'car', 'chevrolet',
       'chrysler', 'dodge', 'ford', 'honda', 'hyndai kia', 'hyundai',
       'infiniti', 'kia', 'lincoln', 'mazda', 'mercedes', 'mercedes benz',
       'mercury', 'mitsubishi', 'nissan', 'pontiac', 'problem', 'saturn',
       'seat', 'sedan', 'subaru', 'suzuki', 'toyata', 'toyota',
       'volkswagen', 'volkwagen', 'volvo'], dtype=object)

In [9]:
brands_edited=['acura', 'audi', 'bmw', 'buick', 'cadillac', 'car', 'chevrolet',
       'chrysler', 'dodge', 'ford', 'honda', 'hyundai kia', 'hyundai',
       'infiniti', 'kia', 'lincoln', 'mazda', 'mercedes',
       'mercury', 'mitsubishi', 'nissan', 'pontiac', 'problem', 'saturn',
       'seat', 'sedan', 'subaru', 'suzuki', 'toyota',
       'volkswagen', 'volvo']
brands=brands_edited

In [10]:
def remove_punctuation(text):
    '''remove all those punctuations
    and add a space before and after each brand name'''
    pun=['.',',',';',':','\\','?','(',')','-','"','/','\'s']
    for p in pun:
        text=text.replace(p,' ')
    for brand in brands:
        text=text.replace(brand,' '+brand+' ')
    return text

posts_nopun=s_post_converted.map(lambda x:remove_punctuation(str(x)))

In [11]:
posts_nopun

0       i personally think that with a few tweaks the ...
1       i am debating a new purchase and these two are...
2       great handling  rwd   hyundai ent engine and t...
3       and no manual tranny  that may not matter to y...
4       one beauty of  bmw  3 series is that there are...
                              ...                        
4995        because the new 3 series is the class leader 
4996    yes  you'll be fine with rwd  even north of da...
4997    pearl  thanks for the tips! i usually don't go...
4998    i agree that the rwd decision is a non issue i...
4999    >but i have to disagree with remark regarding ...
Name: 2, Length: 5000, dtype: object

In [12]:
# first check if a brand in a text and import re
def find_idx(brand, text): 
    '''check if a brand is in a post first,
    find the index of a brand in a post
        '''
    total_count=len(re.findall(brand,text))
    token=text.split()
    idx_temp = token.index(brand)
    idx=idx_temp
    idx_list=[idx]
    count=1
    while idx<len(token) and count<total_count:
        idx_temp = token[idx+1:].index(brand)
        idx=idx+idx_temp+1
        idx_list.append(idx) 
        count+=1
    return idx_list

In [13]:
def features(brand,text):
    '''find feature list for a brand in a post
    using find_idx function to get the index of a brand in a post,
    and then take +5 and -5 words based on these indexes'''
    idx_list=find_idx(brand,text)
    token=text.split()
    feature_list=[]
    for i in idx_list:
        feature_list=feature_list+token[max(0,i-5):i]+token[i+1:min(i+6,len(token))]
    return feature_list

In [14]:
#filterv and filterv2 have the aspirational bigram combinations
#filterv = ['love','hope','want','wish','dream','desire','crave','wanted','always', 'want', 'like', 'kill', 'future', 'aspire']
#filterv1= ['hope','want','wish','dream','desire','crave','buy','purchase','wanted','always']
#filterv2 = ['to','have','car','it','of','this', 'love', 'machine', 'always', 'for']

#filterv and filterv2 have the aspirational bigram combinations
filterv = ['love','hope','want','wish','dream','desire','crave','wanted','always', 'want', 'like', 'kill', 'future', 'aspire']
filterv2 = ['to','have','car','it','of','this', 'love', 'machine', 'always', 'for']

filter_set = set(filterv)
filter_set2 = set(filterv2)

#function to return the occurence of aspirtaional bigrams for every brand
def find(brand,text):
    list_bg = []
    if(brand in text):
        f=features(brand,text)     
        bg = list(zip(*[f[i:] for i in range(2)]))
        for i,j in enumerate(bg):
                if(len(j) != 0):
                     if((j[0] in filter_set) & (j[1] in filter_set2)):
                      #if((j[0] in filter_set)):
                         list_bg.append(j)
        return len(list_bg)

In [15]:
brand_asp = {}
for brand in brands:
    brand_asp[brand] = posts_nopun.map(lambda x: find(brand,x))

In [16]:
#Creating a dataframe to calculate number of posts with aspirations for every brand (asp_brand) 
#and number of posts with aspirations

asp_table = pd.DataFrame(columns = brands)
for brand in brands:
    asp_table[brand]=brand_asp[brand]
asp_table2 = asp_table.fillna(0)

asp_entries = asp_table2.sum(axis=1)
asp_brand = asp_table2[asp_table2 > 0].count(axis = 0)
no_asp = 0
for i in asp_entries.iteritems():
    if(i[1]>0):
        no_asp +=1

In [17]:
#Calculating number of posts with brands mentioned
#Creating a dataframe with required values to calculate lift
import math 

bc = []
aspiration = pd.DataFrame(columns=['brand','asp_brand_count','brand_occurence'])
aspiration['brand'] = brands
for brand in brands:
    count = 0
    for i in asp_table[brand].iteritems():
        if(i[1] is not None):
            if(math.isnan(i[1]) == False):
                count = count + 1
    bc.append(count)
aspiration['asp_brand_count'] = asp_brand.values
aspiration['brand_occurence'] = bc

In [18]:

#Calculate lift for each brand = (n(posts with brand & aspiration)*len(posts))/(n(posts with aspirations)*n(posts with brand))
aspiration['lift'] = (aspiration['asp_brand_count']*len(posts_nopun))/(no_asp * aspiration['brand_occurence'])
print("Top 5 Aspirational brands:")
aspiration.sort_values(by='lift',ascending=False).iloc[:5]

Top 5 Aspirational brands:


,brand,asp_brand_count,brand_occurence,lift
18,mercury,1,8,3.140704
14,kia,1,29,0.866401
16,mazda,3,100,0.753769
22,problem,8,278,0.723040
5,car,74,2701,0.688373
